In [91]:
@file:DependsOn("com.fasterxml.jackson.core:jackson-databind:2.15.2")
import java.io.*
import com.fasterxml.jackson.databind.*


In [92]:
fun readLines(fileName: String): List<String> {
    return File("../data/raw/$fileName").readLines(Charsets.UTF_8)
}

In [93]:
fun sentsToFile(sentences: List<String>, file: String): Unit {
    val fileWriter = FileWriter("../data/formatted/$file")
    fileWriter.use {
        sentences.forEach {
            fileWriter.write(it + System.lineSeparator())
        }
    }
}

In [94]:
fun readAndMapConllu(fileName: String): List<String> {
    val lines = readLines(fileName)
    val relevantLines = lines.filter { it.startsWith("# newdoc") || it.startsWith("# text") }
    return relevantLines
        .map {
            if (it.startsWith("# newdoc")) {
                ""
            } else if (it.startsWith("# text")) {
                it.replace("# text = ", "")
            } else {
                throw RuntimeException("this should not exist: $it")
            }
        }
        // Remove initial empty line
        .drop(1)
}


In [136]:
val mapper = ObjectMapper()
fun readAndMapNer(fileName: String): List<String> {
    val lines = readLines(fileName)
    val sentences: MutableList<String> = lines.map {
        val node = mapper.readTree(it)
        val tokenList = node.path("tokens")
        val sent = tokenList.elements().asSequence()
            .map { it.asText() }
            .toList()
            .joinToString(" ")
            .replace("\\s[,.?!:;]".toRegex()) {
                it.value.trimStart()
            }
            .replace("(\\()(\\s.*\\s)(\\))".toRegex()) {
                it.groupValues[1] + it.groupValues[2].trim() + it.groupValues[3]
            }
            .replace("(\")(\\s.*\\s)(\")".toRegex()) {
                it.groupValues[1] + it.groupValues[2].trim() + it.groupValues[3]
            }

            if (sent.startsWith("\" ")) {
                sent.replaceFirst(" ", "")
            } else {
                sent
            }

    }.toMutableList()

    for (i in sentences.size downTo 0 step 10) {
        if (i > 9 && i < sentences.size) {
            sentences.add(i, "")
        }
    }

    return sentences
}


In [85]:
// val tbTrain = readAndMapConllu("sv_talbanken-ud-train.conllu")
//sentsToFile(tbTrain, "sv_talbanken-se-sent.train.tmp")
// val tbTest = readAndMapConllu("sv_talbanken-ud-test.conllu")
//sentsToFile(tbTest, "sv_talbanken-se-sent.eval")


In [138]:
val sucTrain = readAndMapNer("suc-sprakbanken-cased-train.jsonl")
sentsToFile(sucTrain, "suc-sprakbanken-cased.train")
val sucTest = readAndMapNer("suc-sprakbanken-cased-test.jsonl")
sentsToFile(sucTest, "suc-sprakbanken-cased.eval")

